In [ ]:
import requests
from bs4 import BeautifulSoup 
import re
import psycopg2

In [ ]:
html = requests.get("https://bundestag.de/apps/na/na/fraktion.form?id=338")

In [ ]:
soup = BeautifulSoup(html.content)

In [ ]:
profiles = []
for li in soup.find_all("li"):
    if "id" in li.attrs and li["id"].startswith("abstimmungResult_mdb_"):
        gs = re.search("[^:]+: ([^:]+): (.+)$", li.a.img["alt"])
        name = gs.group(1)
        party = gs.group(2)
        gs = re.search("[^\d]+(\d+)$",li["id"])
        id = gs.group(1)
        
        url = "http://bundestag.de/apps/na/na/fraktion.form?controller=fraktion&id=338&mdbId={}".format(id)
        print(name, party, url)
        
        prof_html = requests.get(url).content
        prof_soup = BeautifulSoup(prof_html)
        prof_url = prof_soup.find_all("div", class_="namentlichPopupText")[0].ul.li.a["href"]
        gs = re.search("//([^?]+)?", prof_url)
        prof_url = "http://www.bundestag.de/{}".format(gs.group(1))
        profiles.append({"name":name, "party":party, "id": id, "url": prof_url})

In [ ]:
profiles

In [ ]:
str = ",".join(["('{}','{}','{}')".format(m["name"],m["party"],m["url"]) for m in profiles])

In [ ]:
con = psycopg2.connect("dbname='data' user='postgres' password='postgres' host='localhost'")

In [ ]:
cur = con.cursor()
cur.execute("insert into temp_tbl (name, party, url) values {}".format(str))
con.commit()

In [ ]:
len(profiles)